# Subset barcode runs
This Python Jupyter notebook is creates a copy of the barcode runs file that contains just specific samples.

First, import Python modules:

In [1]:
import os

import pandas as pd

import yaml

Read in the "master" barcode runs CSV file and the runs used to compute the escape scores for each sample:

In [2]:
with open('../config.yaml') as f:
    config = yaml.safe_load(f)

barcode_runs_file = os.path.join('../', config['barcode_runs'])
print(f"Reading barcode runs from {barcode_runs_file}")
barcode_runs = pd.read_csv(barcode_runs_file)

escape_score_samples_file = os.path.join('../', config['escape_score_samples'])
print(f"Reading escape score samples from {escape_score_samples_file}")
escape_score_samples = pd.read_csv(escape_score_samples_file)

Reading barcode runs from ../data/barcode_runs.csv
Reading escape score samples from ../results/escape_scores/samples.csv


Now read in the samples to subset:

In [3]:
samples_to_subset_df = pd.read_csv('samples_to_subset.csv')
print('Here are the samples we will subset to:')
display(samples_to_subset_df)

samples_to_subset = samples_to_subset_df['sample'].tolist()
assert len(samples_to_subset) == len(set(samples_to_subset)), 'duplicate samples to subset'

Here are the samples we will subset to:


,sample
0,CR3022_400
1,COV2-2677_400
2,COV2-2082_400
3,COV2-2094_400
4,COV2-2165_400
5,COV2-2832_400
6,COV2-2479_400
7,COV2-2050_400
8,COV2-2096_400
9,COV2-2499_400


Now get all the escape-score samples of interest:

In [4]:
if not set(samples_to_subset).issubset(escape_score_samples['name']):
    raise ValueError(f"Not all samples to subset are in the escape score samples.")
    
samples_subset = (
    escape_score_samples
    .query('name in @samples_to_subset')
    .reset_index(drop=True)
    )

print(f"Here are the samples for which we are subsetting barcode runs:")
display(samples_subset)

Here are the samples for which we are subsetting barcode runs:


,name,library,antibody,concentration,concentration_units,date,pre_sample,post_sample,frac_escape,pre_cells_sorted,post_cells_sorted
0,C002_400,lib1,C002,400,ng_per_mL,200923,expt_53-59-none-0-reference,expt_58-C002-400-escape,0.171,1762675.0,NaN
1,C002_400,lib2,C002,400,ng_per_mL,200923,expt_53-59-none-0-reference,expt_58-C002-400-escape,0.179,1740816.0,NaN
2,C105_400,lib1,C105,400,ng_per_mL,200923,expt_53-59-none-0-reference,expt_57-C105-400-escape,0.150,1603797.0,NaN
3,C105_400,lib2,C105,400,ng_per_mL,200923,expt_53-59-none-0-reference,expt_57-C105-400-escape,0.133,1600974.0,NaN
4,C110_400,lib1,C110,400,ng_per_mL,200923,expt_53-59-none-0-reference,expt_56-C110-400-escape,0.135,1455039.0,NaN
5,C110_400,lib2,C110,400,ng_per_mL,200923,expt_53-59-none-0-reference,expt_56-C110-400-escape,0.141,1458602.0,NaN
6,C121_400,lib1,C121,400,ng_per_mL,200923,expt_53-59-none-0-reference,expt_55-C121-400-escape,0.184,2002999.0,NaN
7,C121_400,lib2,C121,400,ng_per_mL,200923,expt_53-59-none-0-reference,expt_55-C121-400-escape,0.176,2013477.0,NaN
8,C135_400,lib1,C135,400,ng_per_mL,200923,expt_53-59-none-0-reference,expt_53-C135-400-escape,0.162,2000000.0,NaN
9,C135_400,lib2,C135,400,ng_per_mL,200923,expt_53-59-none-0-reference,expt_53-C135-400-escape,0.168,2003895.0,NaN


Now unfold the samples of interest into the actual relevant barcode runs (this requires getting both the pre- and post-selection run for each sample):

In [5]:
barcode_runs_subset = (
    samples_subset
    .melt(id_vars=['name', 'library'],
          value_vars=['pre_sample', 'post_sample'],
          value_name='sample',
          var_name='sample_type')
    [['library', 'sample']]
    .drop_duplicates()
    .reset_index(drop=True)
    .merge(barcode_runs,
           how='left',
           on=['library', 'sample'],
           validate='one_to_one',
           )
    )

assert barcode_runs_subset['R1'].notnull().all(), 'some barcode runs missing R1'

barcode_runs_subset_file = 'barcode_runs_subset.csv'
print(f"Here are the subsetted barcode runs. Writing to {barcode_runs_subset_file}")
barcode_runs_subset.to_csv(barcode_runs_subset_file, index=False)
display(barcode_runs_subset)

Here are the subsetted barcode runs. Writing to barcode_runs_subset.csv


,library,sample,date,experiment,antibody,concentration,concentration_units,group,selection,frac_escape,cells_sorted,R1
0,lib1,expt_53-59-none-0-reference,200923,expt_53-59,none,0,ng_per_mL,clinical_serum,reference,NaN,NaN,/shared/ngs/illumina/agreaney/201012_D00300_10...
1,lib2,expt_53-59-none-0-reference,200923,expt_53-59,none,0,ng_per_mL,clinical_serum,reference,NaN,NaN,/shared/ngs/illumina/agreaney/201012_D00300_10...
2,lib1,expt_24-33-none-0-reference,200904,expt_24-33,none,0,ng_per_mL,clinical_serum,reference,NaN,160000000.0,/shared/ngs/illumina/agreaney/200914_D00300_10...
3,lib2,expt_24-33-none-0-reference,200904,expt_24-33,none,0,ng_per_mL,clinical_serum,reference,NaN,160000000.0,/shared/ngs/illumina/agreaney/200914_D00300_10...
4,lib1,expt_60-64-none-0-reference,200929,expt_60-64,none,0,dilution,clinical_serum,reference,NaN,NaN,/shared/ngs/illumina/agreaney/201012_D00300_10...
...,...,...,...,...,...,...,...,...,...,...,...,...
59,lib2,expt_24-REGN10933-400-escape,200904,expt_24,REGN10933,400,ng_per_mL,clinical_serum,escape,0.154,984237.0,/shared/ngs/illumina/agreaney/201012_D00300_10...
60,lib1,expt_26-REGN10933+REGN10987-400-escape,200904,expt_26,REGN10933+REGN10987,400,ng_per_mL,clinical_serum,escape,0.101,635898.0,/shared/ngs/illumina/agreaney/201012_D00300_10...
61,lib2,expt_26-REGN10933+REGN10987-400-escape,200904,expt_26,REGN10933+REGN10987,400,ng_per_mL,clinical_serum,escape,0.083,585188.0,/shared/ngs/illumina/agreaney/201012_D00300_10...
62,lib1,expt_25-REGN10987-400-escape,200904,expt_25,REGN10987,400,ng_per_mL,clinical_serum,escape,0.148,1155004.0,/shared/ngs/illumina/agreaney/201012_D00300_10...
